In [ ]:
from forward import solve_forward
from simulation import Simulation
import numpy as np
import mat73
from net import EEG_CNN
import scipy.io
import matplotlib.pyplot as plt
from matplotlib import cm

In [ ]:
sources = np.load('../downsampled_dipoles-10k/1e-15/sources_10TeD.npy')
eeg = np.load('../downsampled_dipoles-10k/1e-15/eeg_10TeD.npy')
fwd = solve_forward(num_dipoles='10k')
eeg_topos = mat73.loadmat('../downsampled_dipoles-10k/1e-15/eeg_10TeD_topos.mat')['eeg_topos']
eeg_topos = eeg_topos.transpose(2, 0, 1)

sim = Simulation(fwd=fwd, source_data=sources, eeg_data=eeg)
eeg_cnn = EEG_CNN(sim=sim, eeg_topographies=eeg_topos)

eeg_cnn.load_nn('../Best Neural Nets/eeg_cnn_10k_dipoles.h5')

#### Create simulated data (with AWGN) for evaluation

In [ ]:
fwd = solve_forward(num_dipoles='10k')
sim = Simulation(fwd=fwd, source_data=None, eeg_data=None)
snr = 20
eeg_noisy, sources = sim.create_evaluate_dataset(n_samples=5000, snr=snr)

print(eeg_noisy.shape)
# np.save('./eval_sim_data/{}db/eeg_noisy.npy'.format(snr),eeg_noisy)
# np.save('./eval_sim_data/{}db/sources.npy'.format(snr),sources)

#### Eval with noisy sim data 


In [ ]:
# topos were generated with matlab with file create_cnn_input.
snr = 20
eeg_topos = mat73.loadmat('./eval_sim_data/{}db/eeg_topos_noisy.mat'.format(snr))['eeg_topos']
eeg_topos = eeg_topos.transpose(2,0,1)
predicted_sources = eeg_cnn.predict(eeg=eeg_topos).T
np.save('./eval_sim_data/{}db/predicted_sources.npy'.format(snr),predicted_sources)

#### Eval with real data

In [ ]:
ms='20'
real_data_file = './real_data/{}ms/eeg_topo_real_{}ms.mat'.format(ms,ms)
topos_eval =  scipy.io.loadmat(real_data_file)['Zi'] 

plt.contourf(topos_eval, cmap=cm.get_cmap('viridis'))
cbar = plt.colorbar()
plt.title('Topography for {} ms'.format(ms.replace('_','.')))
plt.show()

topos_eval= np.expand_dims(topos_eval, axis=0)

In [ ]:
predicted_sources = eeg_cnn.predict(eeg=topos_eval).T

In [ ]:
np.save('./real_data/{}ms/pred_sources_{}.npy'.format(ms,ms),predicted_sources)